<a href="https://colab.research.google.com/github/AadityaPanchakshikar/Technical-Analysis-w-py/blob/main/Renko%2B_%2Bbrick%2BSize.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Renko

-  Built using price movements
-  Filter Noise
-  New brick gets formed only when the price moves a certain amount
-  Brick Size can be either Fixed Price, Percentage Change or ATR
-  Time scale not fixed
-  Uses close Price

In [ ]:
import pandas as pd
import numpy as np


# Mount Google Drive to access files
from google.colab import drive
drive.mount('/content/drive')

file_path = '/content/drive/MyDrive/Colab Notebooks/Trading/Data/minute.csv'
df = pd.read_csv(file_path)
df.head(2)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,date,open,high,low,close
0,2022-09-09 09:15:00,17913,17925,17911,17918
1,2022-09-09 09:16:00,17917,17918,17902,17905


In [ ]:
brick_size = 5
df2 = pd.DataFrame(columns=['date', 'open', 'high', 'low', 'close'], data=[])
df2.loc[0] = df.iloc[0]
close = df.iloc[0]['close'] // brick_size * brick_size
df2.iloc[0, 1:] = [close - brick_size]
df2['color'] = 'Green'
columns = ['date', 'open', 'high', 'low', 'close', 'color']

for index, row in df.iterrows():
    close = row['close']
    date = row['date']
    last_row = df2.iloc[-1]
    prv_color = last_row['color']
    prv_close = last_row['close']
    bricks = int((close - prv_close) / brick_size)
    data = []
    if prv_color == 'Green' and bricks >= 1:
        color = 'Green'
        for i in range(bricks):
            t = [date, prv_close, prv_close + brick_size, prv_close, prv_close + brick_size, color]
            data.append(t)
            prv_close += brick_size
    elif prv_color == 'Green' and bricks <= -2:
        color = 'Red'
        bricks += 1
        prv_close -= brick_size
        for i in range(abs(bricks)):
            t = [date, prv_close, prv_close, prv_close - brick_size, prv_close - brick_size, color]
            data.append(t)
            prv_close -= brick_size
    elif prv_color == 'Red' and bricks <= -1:
        color = 'Red'
        for i in range(abs(bricks)):
            t = [date, prv_close, prv_close, prv_close - brick_size, prv_close - brick_size, color]
            data.append(t)
            prv_close -= brick_size
    elif prv_color == 'Red' and bricks >= 2:
        color = 'Green'
        bricks -= 1
        prv_close += brick_size
        for i in range(abs(bricks)):
            t = [date, prv_close, prv_close + brick_size, prv_close, prv_close + brick_size, color]
            data.append(t)
            prv_close += brick_size
    else:
        continue
    df3 = pd.DataFrame(data=data, columns=columns)
    df2 = pd.concat([df2, df3])
df2.reset_index(inplace=True, drop=True)

In [ ]:
df2[['date', 'color']].tail(10)

,date,color
99,2022-09-09 15:00:00,Red
100,2022-09-09 15:00:00,Red
101,2022-09-09 15:13:00,Red
102,2022-09-09 15:16:00,Red
103,2022-09-09 15:16:00,Red
104,2022-09-09 15:18:00,Red
105,2022-09-09 15:20:00,Red
106,2022-09-09 15:20:00,Red
107,2022-09-09 15:27:00,Green
108,2022-09-09 15:30:00,Green
